In [1]:
import ujson as json
import os
%matplotlib inline
import utils
import matplotlib.pyplot as plt
from utils import *
from __future__ import division, print_function
from shutil import copyfile
from sklearn.metrics import log_loss
import pandas as pd
from keras.utils import get_file
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.
/home/paperspace/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def extract_color_histogram(image):
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    cv2.normalize(hist,hist)
    return hist.flatten()

In [138]:
# path to grayscale images
train_dir = "../crop_aqb/Train_gray/"  
classes = os.listdir(train_dir)
classes = [x for x in os.listdir(train_dir) if not x.startswith('.')]
print ('The number of classes',classes)

The number of classes ['Fish_3', 'Fish_1', 'Fish_2', 'Fish_5', 'Fish_4']


In [127]:
image_path_list = []
for c in classes:
    fish_images = [train_dir+c+'/'+item for item in os.listdir(train_dir+c+'/')]
    image_path_list.extend(fish_images)
    

In [128]:
labels = []
for c in classes:
    l = [c]*len(os.listdir(train_dir+c+'/'))
    labels.extend(l)

In [129]:
labels = LabelEncoder().fit_transform(labels)

In [130]:
features = []
for i,image_path in enumerate(image_path_list):
    image = cv2.imread(image_path,0)
    hist = extract_color_histogram(image)
    features.append(hist)
    if(i%100==0):
        print(str(i)+ "  completed")


0  completed
100  completed


In [131]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size = 0.20, random_state = 42)

In [132]:
model = KNeighborsClassifier(n_neighbors = 5, n_jobs = -1,weights='distance',metric='cosine')

In [133]:
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='distance')

In [134]:
accuracy = model.score(X_test,y_test)

In [135]:
print (accuracy)

0.75


In [136]:
preds_validation = model.predict_proba(X_test)

In [137]:
log_loss(y_test,preds_validation)

6.675014789910345